In [39]:
%pip install pandas
import pandas as pd

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [46]:
customers_data = pd.read_csv('data_sets/customers.csv')

# CHECK FOR NULL VALUES
print(customers_data.isnull().sum())

# USE x_terminal_id and y_terminal__id to calculate euclidean distance
customers_data['distance'] = ((customers_data['x_customer_id'] - customers_data['y_customer_id'])**2)**0.5

# DROP COLUMNS THAT ARE NOT NEEDED
customers_data = customers_data.drop(columns=['x_customer_id', 'y_customer_id'])

customers_data

CUSTOMER_ID      0
x_customer_id    0
y_customer_id    0
dtype: int64


,CUSTOMER_ID,distance
0,7894622031164826,84.885726
1,65009550197454,61.108141
2,5381378528001886,52.106019
3,344044685095332,54.379830
4,5281794686066290,39.704794
...,...,...
49995,2018185637039049,58.958321
49996,7838600820947425,5.662280
49997,1310171896045522,41.466918
49998,3346506901987756,30.413742


In [61]:
merchants_data = pd.read_csv('data_sets/merchants.csv')

# CHECK FOR NULL VALUES
print(merchants_data.isnull().sum())

# DROP COLUMNS THAT ARE NOT NEEDED
merchants_data = merchants_data.drop(columns=['BUSINESS_TYPE', 'MCC_CODE', 'LEGAL_NAME', 'FOUNDATION_DATE',
                                                'TAX_EXCEMPT_INDICATOR', 'ACTIVE_FROM', 'TRADING_FROM', 'ANNUAL_TURNOVER_CARD',
                                                  'AVERAGE_TICKET_SALE_AMOUNT', 'DELIVERY_SAME_DAYS_PERCENTAGE',
                                                    'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE', 
                                                    'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE'])

# CALCULATE DIFFERENCE IN DEPOSIT_REQUIRED_PERCENTAGE AND DEPOSIT_PERCENTAGE AND SAVE TO DEPOSIT_DIFF
merchants_data['DEPOSIT_DIFF'] = merchants_data['DEPOSIT_REQUIRED_PERCENTAGE'] - merchants_data['DEPOSIT_PERCENTAGE']

# DROP THE INDIVIDUAL DEPOSIT_PERCENTAGE AND DEPOSIT_REQUIRED_PERCENTAGE COLUMNS
merchants_data = merchants_data.drop(columns=['DEPOSIT_REQUIRED_PERCENTAGE', 'DEPOSIT_PERCENTAGE'])

merchants_data

MERCHANT_ID                           0
BUSINESS_TYPE                         0
MCC_CODE                              0
LEGAL_NAME                            0
FOUNDATION_DATE                       0
TAX_EXCEMPT_INDICATOR                 0
OUTLET_TYPE                           0
ACTIVE_FROM                           0
TRADING_FROM                          0
ANNUAL_TURNOVER_CARD                  0
ANNUAL_TURNOVER                       0
AVERAGE_TICKET_SALE_AMOUNT            0
PAYMENT_PERCENTAGE_FACE_TO_FACE       0
PAYMENT_PERCENTAGE_ECOM               0
PAYMENT_PERCENTAGE_MOTO               0
DEPOSIT_REQUIRED_PERCENTAGE           0
DEPOSIT_PERCENTAGE                    0
DELIVERY_SAME_DAYS_PERCENTAGE         0
DELIVERY_WEEK_ONE_PERCENTAGE          0
DELIVERY_WEEK_TWO_PERCENTAGE          0
DELIVERY_OVER_TWO_WEEKS_PERCENTAGE    0
dtype: int64


,MERCHANT_ID,OUTLET_TYPE,ANNUAL_TURNOVER,PAYMENT_PERCENTAGE_FACE_TO_FACE,PAYMENT_PERCENTAGE_ECOM,PAYMENT_PERCENTAGE_MOTO,DEPOSIT_DIFF
0,1b7557bb-208b-4fdc-ba91-eca98298541a,Ecommerce,351840227,49,51,0,0
1,6c948718-ea18-4d07-85d7-0916eb55e699,Ecommerce,411017547,50,50,0,0
2,f0151a49-7ac0-4258-93db-46bee2db6f66,Ecommerce,306887834,54,46,0,0
3,07089d03-0d63-4d24-a3f9-e651fda07cdc,Ecommerce,686209305,57,43,0,0
4,2c16629a-da48-4e69-9862-8fd521a53bbc,Ecommerce,384701644,62,33,5,0
...,...,...,...,...,...,...,...
30447,f58e8478-1566-4d1c-b0cb-b2106c8da3be,Face to Face and Ecommerce,304951,85,15,0,0
30448,68618d14-1e80-449d-95e3-6fa9e61901be,Face to Face and Ecommerce,735472,100,0,0,0
30449,e3cfd7f2-e2d1-40ea-b7b3-2583c57ecb3a,Face to Face and Ecommerce,645184,100,0,0,0
30450,3f8dff6f-d5b0-4963-99fa-3c64830fed23,Face to Face and Ecommerce,912396,100,0,0,0


In [48]:
terminals_data = pd.read_csv('data_sets/terminals.csv')

# CHECK FOR NULL VALUES
print(terminals_data.isnull().sum())

# USE x_terminal_id and y_terminal__id to calculate euclidean distance
terminals_data['distance'] = ((terminals_data['x_terminal_id'] - terminals_data['y_terminal__id'])**2)**0.5

# DROP COLUMNS THAT ARE NOT NEEDED
terminals_data = terminals_data.drop(columns=['x_terminal_id', 'y_terminal__id'])

terminals_data

TERMINAL_ID       0
x_terminal_id     0
y_terminal__id    0
dtype: int64


,TERMINAL_ID,distance
0,9754224,27.353181
1,37467861,40.981413
2,20678215,33.963044
3,12724021,26.240403
4,2989793,29.912286
...,...,...
4995,47999983,5.515555
4996,13129180,16.320659
4997,74645821,29.213743
4998,65145440,1.160258


In [53]:
transactions_test_data = pd.read_csv('data_sets/transactions_test.csv')

# CHECK FOR NULL VALUES
print(transactions_test_data.isnull().sum())

# DROP COLUMNS
transactions_test_data = transactions_test_data.drop(columns=['TX_ID', 'TX_TS', 'TX_AMOUNT', 'FAILURE_CODE',
                                                                'FAILURE_REASON', 'CARD_BRAND', 'CARD_DATA', 'CARD_EXPIRY_DATE',
                                                                  'TRANSACTION_STATUS', 'TRANSACTION_TYPE'])

# DROP COLUMN IF ALL VALUES ARE 0
transactions_test_data = transactions_test_data.loc[:, (transactions_test_data != 0.0).any(axis=0)]

# DROP ROW IF A VALUE IS NAN
transactions_test_data = transactions_test_data.dropna()

# CHECK FOR NULL VALUES AGAIN
print(transactions_test_data.isnull().sum())

transactions_test_data

TX_ID                                        0
TX_TS                                        0
CUSTOMER_ID                                  0
TERMINAL_ID                                  0
TX_AMOUNT                                    0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT        0
TRANSACTION_CASHBACK_AMOUNT                  0
CARD_EXPIRY_DATE                             0
CARD_DATA                                    0
CARD_BRAND                                   0
TRANSACTION_TYPE                             0
TRANSACTION_STATUS                           0
FAILURE_CODE                             70009
FAILURE_REASON                           70009
TRANSACTION_CURRENCY                         0
CARD_COUNTRY_CODE                            0
MERCHANT_ID                                  1
IS_RECURRING_TRANSACTION                     0
ACQUIRER_ID                                  0
CARDHOLDER_AUTH_METHOD                       0
ID_JOIN                                      0
dtype: int64


,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,TRANSACTION_CURRENCY,CARD_COUNTRY_CODE,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN
0,4635360198955981,4641801,34.61,0.0,USD,US,52f53dfb-6f2c-4584-8c82-aeafbcd7a02d,False,ACQ2,Signature,5637.0
1,3695434572951846,75652463,39.74,0.0,CNY,US,ebd8abbf-db80-4ac2-a83b-6b3642f7140a,False,ACQ3,Signature,16253.0
2,2021639056384826,45740229,15.60,0.0,USD,FI,faffd1ae-c962-4a11-b664-2ce691c23df8,False,ACQ3,Signature,20501.0
3,5182312031642618,12372906,62.81,0.0,CNY,US,ef16b77c-1595-4af7-9a50-a0b9f8507bbf,False,ACQ3,Signature,450.0
4,2904034201365313,61785600,71.00,0.0,USD,CA,b62e5e95-63fa-4fa8-b3d2-3eb03616208a,False,ACQ1,Signature,24900.0
...,...,...,...,...,...,...,...,...,...,...,...
71134,7090668360287420,76409951,26.08,0.0,USD,CA,a3c9dbaa-b003-429a-bf20-8df2036f3d95,False,ACQ4,Offline enciphered PIN and signature,5445.0
71135,8394492078987172,22410848,90.94,0.0,USD,DE,5b02c96f-87ab-49e5-bec0-58d284654fa6,False,ACQ4,Offline enciphered PIN and signature,20215.0
71136,2115306202677133,45761959,20.25,0.0,USD,FI,6a8b7002-938a-4fba-829c-7f47a6d3838a,False,ACQ4,Offline enciphered PIN and signature,20895.0
71137,555840843479473,80700449,68.04,0.0,USD,RO,e2de765a-bef0-4793-9517-cc647acc5060,False,ACQ4,Offline enciphered PIN and signature,20954.0


In [55]:
transactions_train_data = pd.read_csv('data_sets/transactions_train.csv')

# CHECK FOR NULL VALUES
print(transactions_train_data.isnull().sum())

# Clean data:
transactions_train_data = transactions_train_data.drop(columns=['TX_ID', 'TX_TS', 'TX_AMOUNT', 'FAILURE_CODE',
                                                                 'FAILURE_REASON', 'CARD_BRAND', 'CARD_DATA',
                                                                   'CARD_EXPIRY_DATE', 'TRANSACTION_STATUS', 'TRANSACTION_TYPE'])

# DROP COLUMN IF ALL VALUES ARE 0
transactions_train_data = transactions_train_data.loc[:, (transactions_train_data != 0.0).any(axis=0)]

# DROP ROW IF A VALUE IS NAN
transactions_train_data = transactions_train_data.dropna()

# CHECK FOR NULL VALUES AGAIN
print(transactions_train_data.isnull().sum())

transactions_train_data

C:\Users\G994623\AppData\Local\Temp\ipykernel_12256\3987224124.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_train_data = pd.read_csv('data_sets/transactions_train.csv')


CUSTOMER_ID                              0
TERMINAL_ID                              0
TX_FRAUD                                 0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT    0
TRANSACTION_CASHBACK_AMOUNT              0
TRANSACTION_CURRENCY                     0
CARD_COUNTRY_CODE                        0
MERCHANT_ID                              0
IS_RECURRING_TRANSACTION                 0
ACQUIRER_ID                              0
CARDHOLDER_AUTH_METHOD                   0
dtype: int64


In [59]:
# MERGE TABLES FOR TRAINING DATA
train_data = pd.merge(transactions_train_data, merchants_data, on='MERCHANT_ID', how='left')

# PRINT ANY ENTRIES THAT ARE NA
print(train_data[train_data.isna().any(axis=1)])

train_data

Empty DataFrame
Columns: [CUSTOMER_ID, TERMINAL_ID, TX_FRAUD, TRANSACTION_GOODS_AND_SERVICES_AMOUNT, TRANSACTION_CASHBACK_AMOUNT, TRANSACTION_CURRENCY, CARD_COUNTRY_CODE, MERCHANT_ID, IS_RECURRING_TRANSACTION, ACQUIRER_ID, CARDHOLDER_AUTH_METHOD, OUTLET_TYPE, ANNUAL_TURNOVER, PAYMENT_PERCENTAGE_FACE_TO_FACE, PAYMENT_PERCENTAGE_ECOM, PAYMENT_PERCENTAGE_MOTO, DEPOSIT_REQUIRED_PERCENTAGE, DEPOSIT_PERCENTAGE]
Index: []


,CUSTOMER_ID,TERMINAL_ID,TX_FRAUD,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,TRANSACTION_CURRENCY,CARD_COUNTRY_CODE,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,OUTLET_TYPE,ANNUAL_TURNOVER,PAYMENT_PERCENTAGE_FACE_TO_FACE,PAYMENT_PERCENTAGE_ECOM,PAYMENT_PERCENTAGE_MOTO,DEPOSIT_REQUIRED_PERCENTAGE,DEPOSIT_PERCENTAGE
0,7323088226725338,55875360,0,0.00,0.0,USD,SA,136def43-2ef3-4bc9-a8c2-60598c316e9c,False,ACQ1,Online PIN,Face to Face,23598,37,63,0,69,69
1,8627200351202456,31418810,0,1.14,0.0,USD,CA,a019e1d4-a3b3-479e-9747-a2ece27ff980,False,ACQ2,Signature,Face to Face,340735,84,16,0,15,25
2,8983139483148479,84302882,0,0.46,0.0,USD,CA,f15d1eca-36f0-4f11-b17d-fe36475b021e,False,ACQ1,Online PIN,Face to Face and Ecommerce,707188,79,21,0,49,49
3,2016744441942099,26364082,0,2.06,0.0,GBP,US,7bd1dea7-72fa-41b3-8232-b315e1187b58,False,ACQ2,Offline plaintext PIN,Face to Face and Ecommerce,104869,66,34,0,22,22
4,6888146658050582,34582161,0,1.50,0.0,USD,BE,eee6cb9f-b64d-4710-8c36-1c2b426d7466,False,ACQ2,Offline plaintext PIN,Face to Face and Ecommerce,788913,43,57,0,17,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066434,2426772567661946,29332740,0,18.54,0.0,GBP,US,26edbb0d-e15c-4e05-a36a-edba7a80348e,False,ACQ2,Online PIN,Face to Face,166262,80,20,0,17,17
1066435,5935188564996005,17853512,0,49.26,0.0,USD,US,badaefe1-3194-4c8d-bf17-1170a7434b91,False,ACQ3,Offline enciphered PIN,Face to Face and Ecommerce,398860,65,35,0,70,70
1066436,1236547395079298,47342746,0,49.26,0.0,EUR,BE,f5ea7e47-61b5-43a2-acaa-4ac66a0d527f,False,ACQ3,Online PIN,Face to Face and Ecommerce,911287,57,43,0,24,24
1066437,1023056874020037,35501969,0,79.98,0.0,USD,DE,6e9e0952-1b76-45d5-8693-a4423d0d80da,True,ACQ4,Online PIN,Face to Face,425738,76,24,0,20,20


In [58]:
# MERGE TABLES FOR TESTING DATA
test_data = pd.merge(transactions_test_data, merchants_data, on='MERCHANT_ID', how='left')

# PRINT ANY ENTRIES THAT ARE NA
print(test_data[test_data.isna().any(axis=1)])

test_data

Empty DataFrame
Columns: [CUSTOMER_ID, TERMINAL_ID, TRANSACTION_GOODS_AND_SERVICES_AMOUNT, TRANSACTION_CASHBACK_AMOUNT, TRANSACTION_CURRENCY, CARD_COUNTRY_CODE, MERCHANT_ID, IS_RECURRING_TRANSACTION, ACQUIRER_ID, CARDHOLDER_AUTH_METHOD, ID_JOIN, OUTLET_TYPE, ANNUAL_TURNOVER, PAYMENT_PERCENTAGE_FACE_TO_FACE, PAYMENT_PERCENTAGE_ECOM, PAYMENT_PERCENTAGE_MOTO, DEPOSIT_REQUIRED_PERCENTAGE, DEPOSIT_PERCENTAGE]
Index: []


,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,TRANSACTION_CURRENCY,CARD_COUNTRY_CODE,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN,OUTLET_TYPE,ANNUAL_TURNOVER,PAYMENT_PERCENTAGE_FACE_TO_FACE,PAYMENT_PERCENTAGE_ECOM,PAYMENT_PERCENTAGE_MOTO,DEPOSIT_REQUIRED_PERCENTAGE,DEPOSIT_PERCENTAGE
0,4635360198955981,4641801,34.61,0.0,USD,US,52f53dfb-6f2c-4584-8c82-aeafbcd7a02d,False,ACQ2,Signature,5637.0,Ecommerce,389840,39,61,0,17,17
1,3695434572951846,75652463,39.74,0.0,CNY,US,ebd8abbf-db80-4ac2-a83b-6b3642f7140a,False,ACQ3,Signature,16253.0,Face to Face,643907,60,40,0,52,52
2,2021639056384826,45740229,15.60,0.0,USD,FI,faffd1ae-c962-4a11-b664-2ce691c23df8,False,ACQ3,Signature,20501.0,Face to Face,560326,78,22,0,66,66
3,5182312031642618,12372906,62.81,0.0,CNY,US,ef16b77c-1595-4af7-9a50-a0b9f8507bbf,False,ACQ3,Signature,450.0,Face to Face,502041340,83,17,0,92,92
4,2904034201365313,61785600,71.00,0.0,USD,CA,b62e5e95-63fa-4fa8-b3d2-3eb03616208a,False,ACQ1,Signature,24900.0,Face to Face,576342,80,20,0,81,81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71133,7090668360287420,76409951,26.08,0.0,USD,CA,a3c9dbaa-b003-429a-bf20-8df2036f3d95,False,ACQ4,Offline enciphered PIN and signature,5445.0,Face to Face and Ecommerce,711524,62,38,0,16,16
71134,8394492078987172,22410848,90.94,0.0,USD,DE,5b02c96f-87ab-49e5-bec0-58d284654fa6,False,ACQ4,Offline enciphered PIN and signature,20215.0,Ecommerce,815031,80,20,0,65,65
71135,2115306202677133,45761959,20.25,0.0,USD,FI,6a8b7002-938a-4fba-829c-7f47a6d3838a,False,ACQ4,Offline enciphered PIN and signature,20895.0,Face to Face,70252,46,54,0,68,68
71136,555840843479473,80700449,68.04,0.0,USD,RO,e2de765a-bef0-4793-9517-cc647acc5060,False,ACQ4,Offline enciphered PIN and signature,20954.0,Face to Face,614036,56,44,0,68,68
